### 1. 载入数据

In [38]:
from sklearn.feature_extraction.text import TfidfTransformer    
from sklearn.feature_extraction.text import CountVectorizer  
from sklearn.feature_extraction.text import HashingVectorizer

In [39]:
corpus = []
for line in open('test.txt', 'r').readlines():
    corpus.append(line.strip())

In [40]:
for i, cor in enumerate(corpus):
    print('{}\t{}'.format(i, cor))

0	新春 备 年货 ， 新年 联欢晚会
1	新春 节目单 ， 春节 联欢晚会 红火
2	大盘 下跌 股市 散户
3	下跌 股市 赚钱
4	金猴 新春 红火 新年
5	新车 新年 年货 新春
6	股市 反弹 下跌
7	股市 散户 赚钱
8	新年 , 看 春节 联欢晚会
9	大盘 下跌 散户


In [41]:
# 将语料转换为词频矩阵，矩阵元素a[i][j]表示文档i中词j的词频
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)
# 获取词袋模型中的所有词语
vocab = vectorizer.get_feature_names()
weight = X.toarray()

In [42]:
print('vocab:\n {}\n'.format(vocab))
print('word freq matrix:\n {}\n'.format(weight))

vocab:
 ['下跌', '反弹', '大盘', '年货', '散户', '新年', '新春', '新车', '春节', '红火', '联欢晚会', '股市', '节目单', '赚钱', '金猴']

word freq matrix:
 [[0 0 0 1 0 1 1 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 1 0 1 1 1 0 1 0 0]
 [1 0 1 0 1 0 0 0 0 0 0 1 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 1 0 1 0]
 [0 0 0 0 0 1 1 0 0 1 0 0 0 0 1]
 [0 0 0 1 0 1 1 1 0 0 0 0 0 0 0]
 [1 1 0 0 0 0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 1 0 1 0]
 [0 0 0 0 0 1 0 0 1 0 1 0 0 0 0]
 [1 0 1 0 1 0 0 0 0 0 0 0 0 0 0]]



### 2. 训练模型
其中设置2个主题，500次迭代

In [43]:
import numpy as np
import lda

In [44]:
model = lda.LDA(n_topics=2, n_iter=500, random_state=1)
model.fit(np.asarray(weight))

INFO:lda:n_documents: 10
INFO:lda:vocab_size: 15
INFO:lda:n_words: 36
INFO:lda:n_topics: 2
INFO:lda:n_iter: 500
INFO:lda:<0> log likelihood: -192
INFO:lda:<10> log likelihood: -140
INFO:lda:<20> log likelihood: -133
INFO:lda:<30> log likelihood: -133
INFO:lda:<40> log likelihood: -133
INFO:lda:<50> log likelihood: -133
INFO:lda:<60> log likelihood: -133
INFO:lda:<70> log likelihood: -133
INFO:lda:<80> log likelihood: -137
INFO:lda:<90> log likelihood: -133
INFO:lda:<100> log likelihood: -133
INFO:lda:<110> log likelihood: -133
INFO:lda:<120> log likelihood: -133
INFO:lda:<130> log likelihood: -133
INFO:lda:<140> log likelihood: -133
INFO:lda:<150> log likelihood: -133
INFO:lda:<160> log likelihood: -133
INFO:lda:<170> log likelihood: -133
INFO:lda:<180> log likelihood: -133
INFO:lda:<190> log likelihood: -133
INFO:lda:<200> log likelihood: -133
INFO:lda:<210> log likelihood: -133
INFO:lda:<220> log likelihood: -133
INFO:lda:<230> log likelihood: -133
INFO:lda:<240> log likelihood: -133

### 3. 主题 - 单词（topic-word）分布

计算vocab[:3]==> 下跌、反弹、大盘 这三个单词在两个topic中的比重。

可以看到下跌、反弹、大盘等股市主题词汇在主题1中的比重更大。

In [45]:
topic_word = model.topic_word_
print('words: {}\n'.format(vocab[:3]))
print('topic_word shape: {}\n'.format(topic_word.shape))
print(topic_word[:, :3])

words: ['下跌', '反弹', '大盘']

topic_word shape: (2, 15)

[[ 0.00049628  0.00049628  0.00049628]
 [ 0.24829721  0.0625387   0.1244582 ]]


计算每个主题中比重最大的前5个词语

In [46]:
n = 5

for i, topic_dist in enumerate(topic_word):
    topic_words = np.array(vocab)[np.argsort(topic_dist)][:n]
    print(topic_words)

['下跌' '反弹' '大盘' '散户' '股市']
['年货' '新年' '新春' '新车' '春节']


### 4. 文档 - 主题（document-topic）分布

计算前三篇文档在两个topic中的比重。

前三篇文档分别为：
- [春节] 新春 备 年货 ， 新年 联欢晚会
- [春节] 新春 节目单 ， 春节 联欢晚会 红火
- [股市] 大盘 下跌 股市 散户

可以看到第一篇和第二篇文档在主题0[春节]的比重大，第三篇文档在主题1[股市]的比重大。

In [47]:
doc_topic = model.doc_topic_
for i, cor in enumerate(corpus[:3]):
    print('{}  {}\t'.format(i, cor))
print('\n')
print('doc_topic shape: {}'.format(doc_topic.shape))
print(doc_topic[:3, :])

0  新春 备 年货 ， 新年 联欢晚会	
1  新春 节目单 ， 春节 联欢晚会 红火	
2  大盘 下跌 股市 散户	


doc_topic shape: (10, 2)
[[ 0.97619048  0.02380952]
 [ 0.98076923  0.01923077]
 [ 0.02380952  0.97619048]]
